In [9]:
import os
import pprint
#from BCBio import GFF
import pandas

In [10]:
## none of these are currently being used
#baf_gff = 'genomes/bafPKo.gff'
#bb31_gff = 'genomes/bbB31.gff'
#bga_gff = 'genomes/bgaPBi.gff'

In [11]:
print(os.getcwd())

/Users/mf019/bioinformatics/longread_GWAS/group2bb/notebook


In [2]:
lr_pangenome_reference = '/Users/mf019/bioinformatics/longread_GWAS/lipoPredict/output/lr_roary/LR_pan_genome_reference.fa'
sr_pangenome_reference = '/Users/mf019/bioinformatics/longread_GWAS/lipoPredict/output/sr_roary/SR_pan_genome_reference.fa'

In [37]:
# Lets make our group to ID mapping.
# We will use this to map our groups to the correct ID
from re import S
from typing import SupportsIndex


with open(lr_pangenome_reference) as f:
    with open('../output/lr_group_to_ID_mapping.txt', 'w') as out:
        lines = f.readlines()
        groups = []
        for line in lines:
            if line.startswith('>'):
                line = line.replace('>', '').split(' ')
                seqID = line[0]
                groupID = ' '.join(line[1::])
                if 'BB' in groupID:
                    print(1,groupID)
                    groupID = groupID.split('BB')[0::] # whateven
                    print(2,groupID)
                    BBgroupID = 'BB'.join(groupID) # why
                    print(3,BBgroupID)
                    out.write(f'{seqID}\t{BBgroupID}') # ok whatever
                else:
                    out.write(f'{seqID}\t{groupID}')

with open(sr_pangenome_reference) as f:
    with open('../output/sr_group_to_ID_mapping.txt', 'w') as out:
        lines = f.readlines()
        groups = []
        for line in lines:
            if line.startswith('>'):
                line = line.replace('>', '').split(' ')
                seqID = line[0]
                groupID = ' '.join(line[1::])
                if 'BB' in groupID:
                    print(1,groupID)
                    groupID = groupID.split('BB')[0::] # whateven
                    print(2,groupID)
                    BBgroupID = 'BB'.join(groupID) # why
                    print(3,BBgroupID)
                    out.write(f'{seqID}\t{BBgroupID}') # ok whatever
                else:
                    out.write(f'{seqID}\t{groupID}')
    #print(groups)

1 Uncharacterized protein BB_0001

2 ['Uncharacterized protein ', '_0001\n']
3 Uncharacterized protein BB_0001

1 Uncharacterized protein BB_0007

2 ['Uncharacterized protein ', '_0007\n']
3 Uncharacterized protein BB_0007

1 YbbR-like domain-containing protein BB-0009

2 ['YbbR-like domain-containing protein ', '-0009\n']
3 YbbR-like domain-containing protein BB-0009

1 Uncharacterized protein BB_0011

2 ['Uncharacterized protein ', '_0011\n']
3 Uncharacterized protein BB_0011

1 Uncharacterized protein BB_0013

2 ['Uncharacterized protein ', '_0013\n']
3 Uncharacterized protein BB_0013

1 Uncharacterized protein BB_0019

2 ['Uncharacterized protein ', '_0019\n']
3 Uncharacterized protein BB_0019

1 Uncharacterized protein BB_0027

2 ['Uncharacterized protein ', '_0027\n']
3 Uncharacterized protein BB_0027

1 Uncharacterized protein BB_0038

2 ['Uncharacterized protein ', '_0038\n']
3 Uncharacterized protein BB_0038

1 Uncharacterized protein BB_0039

2 ['Uncharacterized protein ', '_

In [41]:
# set blast results
lr_blast_results = '../LR_pan_genome_blast.topHit.tsv'
sr_blast_results = '../SR_pan_genome_blast.topHit.tsv'

In [43]:
#Fields="query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score"
results = pandas.read_csv(lr_blast_results, sep = '\t', header=None) # read in blast results .tsv

#create ID to Group translation table
#ID2GROUP.tsv is created by sed on the pan-genome reference fasta # created above......
id2group = pandas.read_csv('../output/lr_group_to_ID_mapping.txt', sep = '\t', header=None)
id2group = pandas.DataFrame({"ID":id2group[0],"group":id2group[1]})

#create a dataframe containing group, geneID, %identity, alignment length, and E-score
hits = pandas.DataFrame({"ID":results[0],
                         "gene":results[1],
                         "percent_ident":results[2],
                         "alignment_length": results[3],
                         "E-score": results[10]})
hits = hits.drop_duplicates() # I have no idea why blast output duplicates for each hit.... drop them
ids = list(hits['ID'].unique()) # pull out unique ID numbers to filter the list by.
hits = hits.loc[(hits["ID"].isin(ids)) & (hits["gene"].str.startswith("gene-BB"))] # filter list by IDs then pull out genes starting with gene-BB.
hits = hits.merge(id2group,how='left') #merge em together
####
#
# To make the gene names easier to work with, use str method to remove prefix 'gene-'. I originally had it set to automatically strip that but due to the
# GFF/annotations having that same prefix, opted to leave it included for easier parsing/comparison. It should probably be removed.
#
####
translatedhits = hits.iloc[:,[0,5,1,2,3,4]] #reorder dataframe
#print(translatedhits) #make sure it isn't broken.

translatedhits.to_csv("../output/lr_groupID_to_BBgene.tsv", sep='\t',header=True,index=None) #write to tsv

In [44]:
#Fields="query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score"
results = pandas.read_csv(sr_blast_results, sep = '\t', header=None) # read in blast results .tsv

#create ID to Group translation table
#ID2GROUP.tsv is created by sed on the pan-genome reference fasta # created above......
id2group = pandas.read_csv('../output/sr_group_to_ID_mapping.txt', sep = '\t', header=None)
id2group = pandas.DataFrame({"ID":id2group[0],"group":id2group[1]})

#create a dataframe containing group, geneID, %identity, alignment length, and E-score
hits = pandas.DataFrame({"ID":results[0],
                         "gene":results[1],
                         "percent_ident":results[2],
                         "alignment_length": results[3],
                         "E-score": results[10]})
hits = hits.drop_duplicates() # I have no idea why blast output duplicates for each hit.... drop them
ids = list(hits['ID'].unique()) # pull out unique ID numbers to filter the list by.
hits = hits.loc[(hits["ID"].isin(ids)) & (hits["gene"].str.startswith("gene-BB"))] # filter list by IDs then pull out genes starting with gene-BB.
hits = hits.merge(id2group,how='left') #merge em together
####
#
# To make the gene names easier to work with, use str method to remove prefix 'gene-'. I originally had it set to automatically strip that but due to the
# GFF/annotations having that same prefix, opted to leave it included for easier parsing/comparison. It should probably be removed.
#
####
translatedhits = hits.iloc[:,[0,5,1,2,3,4]] #reorder dataframe
#print(translatedhits) #make sure it isn't broken.

translatedhits.to_csv("../output/sr_groupID_to_BBgene.tsv", sep='\t',header=True,index=None) #write to tsv

In [ ]:
## parse GTF file
#record = {}
#in_handle = open(bb31_gff)
#for rec in GFF.parse(in_handle):
#    record = rec.features
#for i in enumerate(record):
#    pprint.pprint(record[i[0]].qualifiers)